# Crypto Portfolio Optimization

Goal: Create a machine learning-driven portfolio optimization system for cryptocurrencies, dynamically adjusting allocations to maximize returns and minimize risks based on predictive analytics.

### Top crypto cryptocurrencies 

In [1]:
import requests
import pandas as pd

##### Coingecko API to get the top 20 cryptos 

In [2]:
#&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc

In [3]:
def fetch_top_coins_tickers(vs_currency='usd', limit=30):
    url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc'
    params = {
        'vs_currency': vs_currency,
        'order': 'market_cap_desc',
        'per_page': limit,
        'page': 1,
        'sparkline': False,
        'price_change_percentage': '24h'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching data: Status code {response.status_code}")
        return []

    data = response.json()

##Extract tickers
    tickers = [coin['symbol'].upper() for coin in data]
    return tickers

In [4]:
top_coins = fetch_top_coins_tickers(vs_currency='usd', limit=20)
print(top_coins)

['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'XRP', 'STETH', 'USDC', 'ADA', 'AVAX', 'DOGE', 'TRX', 'LINK', 'DOT', 'UNI', 'MATIC', 'WBTC', 'TON', 'ICP', 'SHIB']


##### yfinance (historical data)

In [5]:
import yfinance as yf


# Specify the symbols for the cryptocurrencies you are interested in
symbols = [f"{coin}-USD" for coin in top_coins]  


df = pd.DataFrame()

# Specify the date range
start_date = '2020-9-1'
end_date = '2024-02-21'

# Fetch the historical data for each symbol
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    data = yf.download(symbol, start=start_date, end=end_date)
    if not data.empty:
        data['Symbol'] = symbol.replace('-USD', '')  
        df = pd.concat([df, data], axis=0)


df.reset_index(inplace=True)




c:\Users\Ethan Vaz Falcao\anaconda3\lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


Fetching data for BTC-USD...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching data for ETH-USD...
Fetching data for USDT-USD...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching data for WBTC-USD...
Fetching data for TON-USD...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching data for ICP-USD...
Fetching data for SHIB-USD...


[*********************100%%**********************]  1 of 1 completed


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24934 entries, 0 to 24933
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       24934 non-null  datetime64[ns]
 1   Open       24934 non-null  float64       
 2   High       24934 non-null  float64       
 3   Low        24934 non-null  float64       
 4   Close      24934 non-null  float64       
 5   Adj Close  24934 non-null  float64       
 6   Volume     24934 non-null  int64         
 7   Symbol     24934 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.5+ MB


### Crypto History Data - data cleansing 

In [7]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.rename(columns={'Volume': 'Volume($)'})

In [8]:
#df['date'] = df['Date'].combine_first(df['index']).combine_first(df['Date'])
#df.drop(['index','Date'], axis=1, inplace=True)
#df=df[['date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Market Cap']]        

In [9]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume($)    0
Symbol       0
dtype: int64

In [10]:
# Convert the 'Date' column to datetime format for proper sorting
df['Date'] = pd.to_datetime(df['Date'])

# Calculate daily returns for each cryptocurrency
df['Daily Return'] = df.groupby('Symbol')['Close'].pct_change()

In [11]:
df.isnull().sum()

Date              0
Open              0
High              0
Low               0
Close             0
Adj Close         0
Volume($)         0
Symbol            0
Daily Return    205
dtype: int64

In [12]:
df_filtered_date = df[df['Date'] == '2024-01-01']

# Now sort this filtered DataFrame based on the 'Close' column to rank the cryptocurrencies
df_sorted_by_close = df_filtered_date.sort_values(by='Close', ascending=False)

# The ranking is implicit in the sorted DataFrame - the first row has the highest closing price and so on
df_sorted_by_close[['Symbol', 'Close']]  # Displaying only the relevant columns for clarity


,Symbol,Close
1217,BTC,44167.332031
21371,WBTC,44080.980469
2485,ETH,2352.327881
8712,STETH,2340.830322
5021,BNB,314.408295
6289,SOL,109.508682
12495,AVAX,41.864464
16299,LINK,15.535809
23656,ICP,12.962581
17567,DOT,8.598925


In [13]:
rows_with_null = df[df.isnull().any(axis=1)]

rows_with_null

,Date,Open,High,Low,Close,Adj Close,Volume($),Symbol,Daily Return
0,2020-09-01,11679.316406,12067.081055,11601.128906,11970.478516,11970.478516,27311555343,BTC,NaN
1268,2020-09-01,434.874451,487.211884,432.079193,477.051910,477.051910,18862763756,ETH,NaN
2536,2020-09-01,1.003056,1.006176,0.997956,1.003957,1.003957,49809584105,USDT,NaN
3804,2020-09-01,23.185047,25.405396,22.988291,24.822096,24.822096,450200844,BNB,NaN
5072,2020-09-01,4.781050,4.944731,4.344516,4.344516,4.344516,16944927,SOL,NaN
...,...,...,...,...,...,...,...,...,...
23888,2021-04-11,0.000000,0.000000,0.000000,0.000000,0.000000,6995583,SHIB,NaN
23889,2021-04-12,0.000000,0.000000,0.000000,0.000000,0.000000,17561875,SHIB,NaN
23890,2021-04-13,0.000000,0.000000,0.000000,0.000000,0.000000,13314013,SHIB,NaN
23891,2021-04-14,0.000000,0.000000,0.000000,0.000000,0.000000,6349137,SHIB,NaN


In [14]:
df['Symbol'].unique()

array(['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'XRP', 'STETH', 'USDC', 'ADA',
       'AVAX', 'DOGE', 'TRX', 'LINK', 'DOT', 'UNI', 'MATIC', 'WBTC',
       'TON', 'ICP', 'SHIB'], dtype=object)

In [15]:
# removed wrapped coins and the stable coin 'usdc' as they won't help in diversify  my portfolio 
# Define the symbols to exclude
exclude_symbols = ['SHIB', 'ICP','STETH','WBTC','USDC','UNI']

# Filter the DataFrame to exclude the defined symbols
df = df[~df['Symbol'].isin(exclude_symbols)]


In [16]:
df['Symbol'].unique()

array(['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'XRP', 'ADA', 'AVAX', 'DOGE',
       'TRX', 'LINK', 'DOT', 'MATIC', 'TON'], dtype=object)

In [17]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume($),Symbol,Daily Return
0,2020-09-01,11679.316406,12067.081055,11601.128906,11970.478516,11970.478516,27311555343,BTC,NaN
1,2020-09-02,11964.823242,11964.823242,11290.793945,11414.034180,11414.034180,28037405299,BTC,-0.046485
2,2020-09-03,11407.191406,11443.022461,10182.464844,10245.296875,10245.296875,31927261555,BTC,-0.102395
3,2020-09-04,10230.365234,10663.919922,10207.940430,10511.813477,10511.813477,29965130374,BTC,0.026014
4,2020-09-05,10512.530273,10581.571289,9946.675781,10169.567383,10169.567383,44916565292,BTC,-0.032558


In [18]:
df.to_csv('data/top_crypto.csv', index=False)